### This Note book is dedicated to compute Graphs and Testing

In [1]:
import os
import numpy as np
import os
import numpy as np
import scipy
import nibabel as nib
from nilearn import image

In [2]:
## Correlation between annotations

def cosine_between_annotations(annotations1, annotations2):
    """
    Compute the correlation between two annotations numpy.
    """
    if len(annotations1)>len(annotations2):
        annotations1 = annotations1[:len(annotations2)]
    else:
        annotations2 = annotations2[:len(annotations1)]
    if len(annotations1.shape)>1:
        annotations1 = annotations1.flatten()
    if len(annotations2.shape)>1:
        annotations2 = annotations2.flatten()
    cosine = scipy.spatial.distance.cosine(annotations1, annotations2)
    pearson = scipy.stats.pearsonr(annotations1, annotations2)[0]
    correlation = np.corrcoef(annotations1.flatten(), annotations2.flatten())[0, 1]
    return cosine ,len(annotations1),len(annotations2)

In [3]:
anot1 = np.load(r"C:\uni\Msc Brain\Lab work\STS_sherlock\projects data\annotations\social_speak_music.npy")
anot2 = np.load(r"C:\uni\Msc Brain\Lab work\STS_sherlock\projects data\annotations\llava_social_speak_full.npy")
anot3 = np.load(r"C:\uni\Msc Brain\Lab work\STS_sherlock\projects data\annotations\llava_social_speak_final.npy")
leyla = np.load(r"C:\uni\Msc Brain\Lab work\STS_sherlock\projects data\annotations\social_nonsocial_convert.npy")

In [4]:
print(cosine_between_annotations(anot1, leyla))
print(cosine_between_annotations(anot2, leyla))
print(cosine_between_annotations(anot3, leyla))


(0.18374806145989708, 1975, 1975)
(0.13713258081295343, 1976, 1976)
(0.1418237905340537, 1053, 1053)
